In [1]:
from preprocessor import Abbreviations
from translator import HuggingFace
from language import Language
from loader import ExcelLoader
from preprocessor import EntityExtractor

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Preporation
Translation, abbreviation replacement, entity extractaction

In [2]:
model_translator=HuggingFace(Language.GERMAN, Language.ENGLISH)

In [3]:
import pandas as pd

In [4]:
data=list(ExcelLoader("/workspaces/de.uke.iam.automapping/src/automapping/VM_208 (3).xlsx", 'sm5', 'Langname', Language.GERMAN))

In [5]:
translated_phrases=model_translator.translate(data, Abbreviations.load_abbreviations('/workspaces/de.uke.iam.automapping/src/automapping/german_abbreviation.xlsx', 'Abbreviation', 'Description'))

In [6]:
translated_phrases

[('HCH_ANAM0021', 'Is there any known hypertension diagnosed by your doctor?'),
 ('HCH_VAEX0004', 'Performance Riva-Rocci - Measurement of left upper arm'),
 ('HCH_VAEX0005', 'Performance Riva-Rocci - Measurement of right upper arm'),
 ('HCH_VAEX0006',
  'Execution Closure pressure measurement left ankle A. tibialis anterior (ATA)'),
 ('HCH_VAEX0007',
  'Execution Closure pressure measurement left ankle A. tibialis posterior (adenosine triphosphate)'),
 ('HCH_VAEX0013', 'Closure pressure left ankle ATA'),
 ('HCH_VAEX0014', 'Closure pressure left ankle ATP'),
 ('HCH_VAEX0016', 'Closure pressure right ankle ATA'),
 ('HCH_VAEX0017',
  'Execution Closure pressure measurement right ankle A. tibialis posterior (adenosine triphosphate)'),
 ('HCH_VAEX0018', 'Closure pressure right ankle ATP'),
 ('HCH_VAEX0019',
  'Execution Closure pressure measurement right ankle A. tibialis anterior (ATA)'),
 ('HCH_VAEX0022', 'Signal quality left ATP'),
 ('HCH_VAEX0023', 'Signal quality right ATP'),
 ('HCH_V

In [7]:
model_entity=EntityExtractor()

In [8]:
prep_data=model_entity(translated_phrases)

In [9]:
list_of_prep_data=list(prep_data)

In [10]:
list_of_prep_data

[('HCH_ANAM0021', 'know hypertension diagnose'),
 ('HCH_VAEX0004', 'performance riva rocci measurement left upper arm'),
 ('HCH_VAEX0005', 'performance riva rocci measurement right upper arm'),
 ('HCH_VAEX0006',
  'execution closure pressure measurement leave ankle a. tibialis anterior ata'),
 ('HCH_VAEX0007',
  'execution closure pressure measurement leave ankle a. tibialis posterior adenosine triphosphate'),
 ('HCH_VAEX0013', 'closure pressure leave ankle ata'),
 ('HCH_VAEX0014', 'closure pressure leave ankle atp'),
 ('HCH_VAEX0016', 'closure pressure right ankle ata'),
 ('HCH_VAEX0017',
  'execution closure pressure measurement right ankle a. tibialis posterior adenosine triphosphate'),
 ('HCH_VAEX0018', 'closure pressure right ankle atp'),
 ('HCH_VAEX0019',
  'execution closure pressure measurement right ankle a. tibialis anterior ata'),
 ('HCH_VAEX0022', 'signal quality leave atp'),
 ('HCH_VAEX0023', 'signal quality right atp'),
 ('HCH_VAEX0024', 'signal quality leave ata'),
 ('HC

## Concepts preporation

In [11]:
import pandas as pd
from concept import Concept

In [12]:
concepts = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/CONCEPT.csv', on_bad_lines="skip", delimiter="\t", low_memory=False)

In [13]:
synonyms = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/CONCEPT_SYNONYM.csv', on_bad_lines="skip", delimiter="\t", low_memory=False)

In [14]:
concepts=Concept.concatenate_concept_with_their_synonyms(concepts, synonyms, ['SNOMED'])

/workspaces/de.uke.iam.automapping/src/automapping/concept.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concepts["concept_name"] = (


## Mapping

In [15]:
from mapper import TfIdf
from detections import Predictions

In [16]:
list_of_prep_data

[('HCH_ANAM0021', 'know hypertension diagnose'),
 ('HCH_VAEX0004', 'performance riva rocci measurement left upper arm'),
 ('HCH_VAEX0005', 'performance riva rocci measurement right upper arm'),
 ('HCH_VAEX0006',
  'execution closure pressure measurement leave ankle a. tibialis anterior ata'),
 ('HCH_VAEX0007',
  'execution closure pressure measurement leave ankle a. tibialis posterior adenosine triphosphate'),
 ('HCH_VAEX0013', 'closure pressure leave ankle ata'),
 ('HCH_VAEX0014', 'closure pressure leave ankle atp'),
 ('HCH_VAEX0016', 'closure pressure right ankle ata'),
 ('HCH_VAEX0017',
  'execution closure pressure measurement right ankle a. tibialis posterior adenosine triphosphate'),
 ('HCH_VAEX0018', 'closure pressure right ankle atp'),
 ('HCH_VAEX0019',
  'execution closure pressure measurement right ankle a. tibialis anterior ata'),
 ('HCH_VAEX0022', 'signal quality leave atp'),
 ('HCH_VAEX0023', 'signal quality right atp'),
 ('HCH_VAEX0024', 'signal quality leave ata'),
 ('HC

In [17]:
model_map=TfIdf(concepts)

In [18]:
mapping=model_map(list_of_prep_data)

In [19]:
df=Predictions.to_df(mapping, 5)

In [20]:
df

,SourceID,SourceName,targetConceptName,targetConceptID,targetDomainID,MatchScore
0,CRP,c reactive protein c reactive protein,c-reactive protein normal,4041180,Condition,0.918862
1,CRP,c reactive protein c reactive protein,c-reactive protein measurement,4208414,Measurement,0.905735
2,CRP,c reactive protein c reactive protein,c-reactive protein reaction,4052151,Observation,0.873187
3,CRP,c reactive protein c reactive protein,c-reactive protein abnormal,439937,Condition,0.855597
4,CRP,c reactive protein c reactive protein,plasma c reactive protein,37398420,Measurement,0.849561
...,...,...,...,...,...,...
2305,tagchron_xxx,currently work shiftwork,currently,4034855,Synonym,0.794434
2306,tagchron_xxx,currently work shiftwork,"ileostomy, has currently",4201717,Synonym,0.581730
2307,tagchron_xxx,currently work shiftwork,asthma - currently active,4194289,Condition,0.557551
2308,tagchron_xxx,currently work shiftwork,education currently received,4076231,Observation,0.541365


In [21]:
df.to_excel('2022_12_20_hchs_description_snomed.xlsx')

In [43]:
import pandas as pd
df_translation=pd.DataFrame()

In [44]:
df_translation['German data']=german_data
df_translation['Translated data']=tr 
df_translation['Clean data']=list_of_prep_data


In [45]:
df_translation

,German data,Translated data,Clean data
0,Ist bei Ihnen ein vom Arzt diagnostizierter Hy...,Is there any known hypertension diagnosed by y...,know hypertension diagnose
1,Durchführung RR - Messung linker Oberarm,Performance Riva-Rocci - Measurement of left u...,performance riva rocci measurement left upper arm
2,Durchführung RR - Messung rechter Oberarm,Performance Riva-Rocci - Measurement of right ...,performance riva rocci measurement right upper...
3,Durchführung Verschlussdruckmessung linker Knö...,Execution Closure pressure measurement left an...,execution closure pressure measurement leave a...
4,Durchführung Verschlussdruckmessung linker Knö...,Execution Closure pressure measurement left an...,execution closure pressure measurement leave a...
...,...,...,...
519,Ich stehe auf nach xx Minuten,I get up after xx minutes,xx minute
520,Ich mache zwischendurch ein Nickerchen oder Sc...,I'll take a nap or a nap in between,nap nap
521,Ich mache ein Nickerchen zwischen ... Uhr,I'll take a nap between...,nap
522,... und ... Uhr.,... and ... o'clock.,o'clock


In [47]:
df_translation.to_excel('2022_11_16_HCHS_description_preprocessing.xlsx')